# Đồ án môn học "Lập trình song song ứng dụng"

# Xác định loại bệnh thực vật bằng phương pháp trích xuất dữ liệu và mô hình XGBoost.

## Giảng viên: Phạm Trọng Nghĩa

### Nhóm 14:
- 1712771 - Bùi Thái Tấn Thành
- 1712770 - Trương Thị Lệ Thanh


**Chạy file:** tải thủ công 1 hình (test 4) vào file này.

# Import

In [ ]:
import numpy as np
import cv2 as cv
from numba import njit, jit, cuda
import math
test_image = cv.imread('/content/images/Test_4.jpg')

Downloading...
From: https://drive.google.com/uc?id=1Eg_0c3mvQCu0SpVjLdik1-g1Tuz22ObS
To: /content/plant-pathology-2020-fgvc7.zip
100% 817M/817M [00:07<00:00, 110MB/s]


In [ ]:
!pip install -U xgboost==1.5.1 numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 173.5 MB 8.4 kB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 34.5 MB 13 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


# Cài đặt tuần tự

### Convert color

#### gray

In [ ]:
@njit
def convert_rgb2gray(in_pixels, out_pixels):
    '''
    Convert color image to grayscale image.
    in_pixels : numpy.ndarray with shape=(h, w, 3)
                h, w is height, width of image
                3 is colors with BGR (blue, green, red) order
        Input RGB image
    
    out_pixels : numpy.ndarray with shape=(h, w)
        Output image in grayscale
    '''
    for r in range(len(in_pixels)):
        for c in range(len(in_pixels[0])):
            out_pixels[r, c] = round(in_pixels[r, c, 0] * 0.114 + 
                                in_pixels[r, c, 1] * 0.587 + 
                                in_pixels[r, c, 2] * 0.299)
    return out_pixels

In [ ]:
img_gray = np.empty(test_image.shape[:2])

In [ ]:
%%time
img_gray = convert_rgb2gray(test_image, img_gray)

In [ ]:
%%time
cv_gray = cv.cvtColor(test_image, cv.COLOR_BGR2GRAY)

In [ ]:
np.abs(cv_gray - img_gray).mean()

#### hsv

`BGR2HSV`: 
-  Biểu diễn cách màu sắc xuất hiện dưới ánh sáng
- H - Hue (Bước sóng ưu thế).
- S - Độ bão hòa (Độ tinh khiết / sắc thái của màu).
- V - Giá trị (Cường độ ánh sáng).

Cài đặt theo openCV: https://learnopencv.com/color-spaces-in-opencv-cpp-python/

In [ ]:
@jit
def convert_bgr2hsv(in_pixels, out_pixels):
    '''
    3d & 3d
    '''
    for i in range(len(in_pixels)):
        for j in range(len(in_pixels[0])):
            b,g,r = in_pixels[i,j,0]/255,in_pixels[i,j,1]/255,in_pixels[i,j,2]/255
            S=0
            H=0
            V = max(r,g,b)
            if V!=0:
                S=round((V-min(r,g,b))/V ,3)
            if r==g and g==b:
                H=0
            elif V==r:
                H= 60*(g-b)/(V-min(r,g,b))
            elif V==g:
                H= 120 + 60*(b-r)/(V-min(r,g,b))
            else:
                H= 240 + 60*(r-g)/(V-min(r,g,b))
            if H<0:
                H = H + 360
            out_pixels[i,j,0] = round(H/2,0)
            out_pixels[i,j,1] =  round(255*S,0)
            out_pixels[i,j,2] = 255*V
    return out_pixels

img=cv.imread('/content/images/Test_4.jpg')
o_img =img.copy()
c2 = convert_bgr2hsv(img,o_img)

img=cv.imread('/content/images/Test_4.jpg')
c1 = cv.cvtColor(img, cv.COLOR_BGR2HSV)

print("----------------error----------")
np.mean(np.abs(c2.astype(np.float32)- c1.astype(np.float32))) # up low

In [ ]:
img=cv.imread('/content/images/Test_4.jpg')
o_img =img.copy()

In [ ]:
%%time
c2 = convert_bgr2hsv(img,o_img)

### GaussianBlur

In [ ]:
def get_gaussian_kernel(ksize, sigma):
    '''
    Hàm tạo bộ lọc Gaussian có kích thước shape x shape với các giá trị có độ lệch chuẩn sigma.

    Đầu vào:
    - ksize (int): kích thước bộ lọc
    - sigma (float): độ lệch chuẩn.

    Đầu ra:
    - kernel (np.array): bộ lọc Gaussian

    Tham khảo:
    https://stackoverflow.com/questions/8204645/implementing-gaussian-blur-how-to-calculate-convolution-matrix-kernel
    https://github.com/opencv/opencv/blob/4.x/modules/imgproc/src/smooth.dispatch.cpp
    '''
    if ksize == 1:
        kernel = np.array([1.])
    elif ksize == 3:
        kernel = np.array([0.25, 0.5, 0.25])
    elif ksize == 5:
        kernel = np.array([0.0625, 0.25, 0.375, 0.25, 0.0625])
    elif ksize == 7:
        kernel = np.array([0.03125, 0.109375, 0.21875, 0.21875, 0.21875, 0.109375, 0.03125])
    else:
        if sigma <= 0:
            sigma = 0.3 * ((ksize - 1) * 0.5 - 1) + 0.8

        G = lambda x: math.exp(-(x - (ksize - 1) / 2)**2 / (2 * sigma**2))
        kernel = np.array([G(x) for x in range(ksize)])
        kernel /= kernel.sum()

    return kernel

@njit
def apply_kernel(image, kernel):
    '''
    Hàm thực hiện phép tích chập 2 chiều giữa ảnh image và bộ lọc kernel.
    Tại vùng biên, phần tử lân cận gần nhất được chọn làm phần tử đệm.
    Độ dời khi duyệt là 1.

    Đầu vào:
    - image (np.array): ảnh đầu vào.
    - kernel (np.array): bộ lọc

    Đầu ra:
    - out (np.array): ảnh đã được áp dụng bộ lọc.
    '''
    image = np.atleast_3d(image)
    offset = (kernel.shape[0] // 2, kernel.shape[1] // 2)
    last_row = image.shape[0] - 1
    last_col = image.shape[1] - 1

    out = np.zeros_like(image, np.uint8)
    out_pixel = np.zeros(image.shape[-1], np.float32)

    for out_r in range(image.shape[0]):
        for out_c in range(image.shape[1]):
            for filter_r, r in enumerate(range(out_r - offset[0], out_r + offset[0] + 1)):
                for filter_c, c in enumerate(range(out_c - offset[1], out_c + offset[1] + 1)):
                    in_r = min(max(0, r), last_row)
                    in_c = min(max(0, c), last_col)

                    out_pixel += image[in_r, in_c] * kernel[filter_r, filter_c]

            out[out_r, out_c] = np.floor(out_pixel).astype(np.uint8)
            out_pixel -= out_pixel

    return out

def create_gaussian_filter(ksize, sigmaX, sigmaY=0):
    '''
    Hàm tạo bộ lọc Gauss

    Đầu vào:
    - ksize (tuple): kích thước bộ lọc Gauss, thể hiện số dòng, số cột.
    - sigmaX, sigmaY (float): độ lệch chuẩn cho giá trị bộ lọc theo chiều dọc và ngang.

    Đầu ra
    - kernel (np.array): bộ lọc Gauss
    '''
    kernel_x = get_gaussian_kernel(ksize[0], sigmaX)
    kernel_x = np.expand_dims(kernel_x, 1)

    if sigmaY == 0:
        sigmaY = sigmaX
    kernel_y = get_gaussian_kernel(ksize[1], sigmaY)
    kernel_y = np.expand_dims(kernel_y, 0)

    return kernel_x @ kernel_y

def gaussian_blur(image, ksize, sigmaX, sigmaY=0):
    '''
    Hàm làm mờ ảnh sử dụng bộ lọc Gauss.

    Đầu vào:
    - image (np.array): ảnh đầu vào.
    - ksize (tuple): kích thước bộ lọc Gauss, thể hiện số dòng, số cột.
    - sigmaX, sigmaY (float): độ lệch chuẩn cho giá trị bộ lọc theo chiều dọc và ngang.

    Đầu ra
    - blur (np.array): ảnh kết quả sau khi áp dụng bộ lọc
    '''
    kernel = create_gaussian_filter(ksize, sigmaX, sigmaY)

    out = apply_kernel(image, kernel)
    if len(image.shape) != len(out.shape):
        out.shape = image.shape

    return out

In [ ]:
%%time
img_blur = gaussian_blur(cv_gray, (3,3), 0)

In [ ]:
%%time
cv_blur = cv.GaussianBlur(cv_gray, (3, 3), 0)

In [ ]:
np.abs(img_blur - cv_blur).mean()

### fd_histogram

trong hàm `fd_histogram` có sử dụng [`BGR2HSV`](#scrollTo=DVbuP0zvEYP-&line=5&uniqifier=1)



In [ ]:
def fd_histogram(image):
    bins = 8
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # compute the color histogram
    hist  = cv.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])

    # normalize the histogram
    cv.normalize(hist, hist)

    # return the histogram
    return hist.ravel()

In [ ]:
@njit
def compute_hist(img, hist):
    '''
    Color histogram: thống kê số lần xuất hiện các mức sáng trong ảnh với bins=8, 
                     phạm vi [0,255] cho mỗi kênh màu
    input: 
            img: numpy.ndarray with shape=(h, w, 3)
            hist: numpy.ndarray with shape=(8,8,8)
    '''
    h, w = img.shape[:2] 
    for i in range(h): 
        for j in range(w): 
            x,y,z=img[i][j][0]//32,img[i][j][1]//32,img[i][j][2]//32
            hist[x][y][z] =hist[x][y][z] + 1 
    return hist

def fd_histogram2(image, mask=None):
    # convert the image to HSV color-space
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # compute the color histogram
    hist = np.zeros((8,8,8), np.float32) 
    hist  = compute_hist(image,hist)

    # normalize the histogram
    cv.normalize(hist, hist)
    hist=hist
    # return the histogram
    return hist.ravel()

In [ ]:
%%time
cv_hist_figure = fd_histogram(cv.imread('/content/Test_4.jpg'))

CPU times: user 54.2 ms, sys: 3.06 ms, total: 57.3 ms
Wall time: 67.5 ms


In [ ]:
%%time
hist_figure = fd_histogram2(cv.imread('/content/Test_4.jpg'))

CPU times: user 432 ms, sys: 0 ns, total: 432 ms
Wall time: 433 ms


In [ ]:
np.abs(cv_hist_figure.astype(np.float64)- hist_figure).mean()

0.0

In [ ]:
hist_figure.mean()

0.0098800175

In [ ]:
temp=cv.imread('/content/images/Test_4.jpg')
hist21 = np.zeros((8,8,8), np.float32) 

In [ ]:
%%time
hist21  = compute_hist(temp,hist21)

CPU times: user 9.3 ms, sys: 913 µs, total: 10.2 ms
Wall time: 10.9 ms


### fd_hu_moments

In [ ]:
def fd_hu_moments(gray_image):
    # image = cv.cvtColor(gray_image, cv.COLOR_BGR2GRAY)
    feature = cv.HuMoments(cv.moments(gray_image)).ravel()
    return feature

In [ ]:
@njit
def cvMoments(img,m_,mu_,nu_):
    '''
    input:  ảnh 2D
        m_ mảng 3x3 : M ~ raw Moments
        mu_ 3x3: mu ~ central moments
        nu_ 3x3: nu ~ normalized central moments 
    output: nu_
    '''
    # tính m_
    for i in range(4):
        for j in range(4):
            temp=0
            if (i,j) not in [(1,3),(2,2),(2,3)]:
                for x in range(img.shape[0]):
                    for y in range(img.shape[1]):
                        temp=temp+img[x,y]*(x**i)*(y**j)
                m_[i,j]=temp
            if i==3:
                break

    # xbar ybar
    xbar=m_[1,0]/m_[0,0]
    ybar=m_[0,1]/m_[0,0]

    # tính mu
    mu_[1,1] = m_[1,1] - xbar*m_[0,1]
    mu_[0,2] = m_[2,0] - xbar*m_[1,0]
    mu_[2,0] = m_[0,2] - ybar*m_[0,1]
    mu_[1,2] = m_[2,1] - 2*xbar*m_[1,1] - ybar*m_[2,0] + 2*(xbar**2)*m_[0,1]
    mu_[2,1] = m_[1,2] - 2*ybar*m_[1,1] - xbar*m_[0,2] + 2*(ybar**2)*m_[1,0]
    mu_[0,3] = m_[3,0] - 3*xbar*m_[2,0] + 2*(xbar**2)*m_[1,0]
    mu_[3,0] = m_[0,3] - 3*ybar*m_[0,2] + 2*(ybar**2)*m_[0,1]

    #tính nu  nu_ji = mu_ji / [m00^(((i+j)/2)+1)]
    for i in range(4):
        for j in range(4):
            nu_[i,j] = mu_[i,j]/(m_[0,0]**(((i+j)/2)+1))
 
    return nu_

# HuMoments
@njit
def cvHuMoments(eta,hu_):
    '''
    eta: output của cvMoments
    hu_: Mảng 1D 7 giá trị 
    '''
    hu_[0] =  eta[2][0] + eta[0][2]
    hu_[1] = (eta[2][0] - eta[0][2])**2 + 4*eta[1][1]**2
    hu_[2] = (eta[3][0] - 3*eta[1][2])**2 + (3*eta[2][1] - eta[0][3])**2
    hu_[3] = (eta[3][0] + eta[1][2])**2 + (eta[2][1] + eta[0][3])**2
    hu_[4] = (eta[3][0] - 3*eta[1][2])*(eta[3][0] + eta[1][2])*((eta[3][0] + eta[1][2])**2 - 3*(eta[2][1] + eta[0][3])**2) +\
                (3*eta[2][1] - eta[0][3])*(eta[2][1] + eta[0][3])*(3*(eta[3][0] + eta[1][2])**2 - (eta[2][1] + eta[0][3])**2)

    hu_[5] = (eta[2][0] - eta[0][2])*((eta[3][0] + eta[1][2])**2 - (eta[2][1] + eta[0][3])**2) + \
          4*eta[1][1]*(eta[3][0] + eta[1][2])*(eta[2][1] + eta[0][3])

    hu_[6] = (3*eta[2][1] - eta[0][3])*(eta[2][1] + eta[0][3])*(3*(eta[3][0] + eta[1][2])**2-(eta[2][1] + eta[0][3])**2) -\
          (eta[3][0] - 3*eta[1][2])*(eta[1][2] + eta[0][3])*(3*(eta[3][0] + eta[1][2])**2-(eta[2][1] + eta[0][3])**2)

    return hu_

def fd_hu_moments2(gray_image):
    m=np.zeros((4, 4), dtype=np.float64)
    mu=np.zeros((4, 4), dtype=np.float64)
    nu=np.zeros((4, 4), dtype=np.float64)
    hu=np.zeros(7,np.float64)

    feature = cvHuMoments(cvMoments(gray_image, m, mu, nu), hu)
    return feature

In [ ]:
cv_gray = cv.cvtColor(cv.imread('/content/images/Test_4.jpg'), cv.COLOR_BGR2GRAY)

In [ ]:
%%time
cv_hu_moments = fd_hu_moments(cv_gray)

CPU times: user 2.07 ms, sys: 1.03 ms, total: 3.1 ms
Wall time: 7.12 ms


In [ ]:
type(cv_hu_moments[0])

In [ ]:
%%time
hu_moments = fd_hu_moments2(cv_gray)

CPU times: user 2.24 s, sys: 18 ms, total: 2.26 s
Wall time: 2.27 s


In [ ]:
np.abs(cv_hu_moments - hu_moments).mean()

### Nén và lấp khuyết

In [ ]:
@jit
def zipImage(src, zip_x, zip_y, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    for idx in range(0, zip_rs * zip_y, zip_y):
        for jdx in range(0, zip_cs * zip_x, zip_x):
            block_img = src[idx : idx + zip_y, jdx : jdx + zip_x]
            num_pixel = np.sum(block_img > 0)
            if num_pixel >= zip_x*zip_y * ratio:
                block_img[:,:] = 1
            else:
                block_img[:,:] = 0

    return src

@jit
# Hàm dùng để liên kết các ô xung quanh để lấp khuyết sẽ trả ra kết quả là một ma trận mask
def joinNeiboorPixel(src, zip_x, zip_y, mask_size, ratio):
    rs, cs = src.shape
    zip_rs = int(rs / zip_y)
    zip_cs = int(cs / zip_x)
    half = int(mask_size / 2)
    dst = src.copy()
    for idx in range(half , zip_rs - half):
        for jdx in range(half, zip_cs - half):
            start_row_mask = (idx - half ) * zip_y
            end_row_mask = (idx + half + 1) * zip_y
            start_col_mask = (jdx - half) * zip_x
            end_col_mask = (jdx + half + 1) * zip_x
            mask_block = src[start_row_mask : end_row_mask, start_col_mask : end_col_mask]
            block_img = dst[idx * zip_y : (idx + 1) * zip_y, jdx * zip_x : (jdx + 1) * zip_x]
            num_pixel = np.sum(mask_block > 0)
            if num_pixel >= mask_size * mask_size * zip_x * zip_y * ratio:
                block_img[:,:] = 1
    return dst

In [ ]:
    img = cv.imread('/content/images/Test_4.jpg')
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (3, 3), 0)

In [ ]:
%%time
mask_img = zipImage(gray, 16, 16, 0.2)


CPU times: user 977 ms, sys: 20.3 ms, total: 998 ms
Wall time: 996 ms


In [ ]:
%%time
mask_img = joinNeiboorPixel(mask_img, 8, 8, 3, 0.15)

CPU times: user 737 ms, sys: 7.16 ms, total: 744 ms
Wall time: 767 ms


### Hàm duyệt cây

In [ ]:
from numpy import array, empty_like, empty, int32, float64, zeros
from math import exp
from queue import Queue
import json
from xgboost import XGBClassifier

In [ ]:
@njit
def softmax(arr:array):
    result = empty_like(arr)
    nrows, ncols = arr.shape

    for i in range(nrows):
        row_sum = 0
        for j in range(ncols):
            element_exp = exp(arr[i, j])
            result[i, j] = element_exp
            row_sum += element_exp

        for j in range(ncols):
            result[i, j] /= row_sum

    return result


def load_tree(feature_arr:array, value_arr:array, tree_dict:dict):
    '''
    Hàm đọc cấu trúc cây ở dạng từ điển.
    Kết quả đọc được lưu vào mảng đặc trưng và giá trị truyền vào.

    Đầu vào:
    - feature_arr, value_arr (numpy.array): mảng lưu đặc trưng và giá trị của các nút trong cây
    - tree_dict (dict): thông tin về cây quyết định lưu ở dạng từ điển
    '''
    # tạo hàng đợi FIFO các nút và chỉ mục tương ứng trong mảng
    tree_q = Queue()
    tree_q.put(tree_dict)

    idx_q = Queue()
    idx_q.put(0)

    while not tree_q.empty():
        node = tree_q.get()
        idx = idx_q.get()

        # xác định nút lá hay nút nhánh
        # nếu có khóa 'leaf' là nút lá
        node_val = node.get('leaf')
        if node_val is None:
            # lưu đặc trưng và giá trị ngưỡng phân nhánh
            feature_arr[idx] = int32(node['split'][1:])
            value_arr[idx] = float64(node['split_condition'])

            # thêm vào hàng đợi thông tin và chỉ mục trong mảng của nút con
            tree_q.put(node['children'][0])
            tree_q.put(node['children'][1])

            idx_q.put(2 * idx + 1)
            idx_q.put(2 * idx + 2)
        else:
            feature_arr[idx] = -1 # giá trị đặc biệt thể hiện nút lá
            value_arr[idx] = float64(node_val)


def load_model(hyperparams_path:str, tree_path:str):
    '''
    Hàm đọc mô hình XGBoost đã lưu.
    Mỗi cây trong mô hình được lưu thành mảng 1 chiều bao gồm 2 mảng đặc trưng và giá trị.
    Mảng giá trị thê hiện giá trị phân nhánh hoặc nút lá tùy giá trị đặc trưng tại ô tương ứng.

    Đầu vào:
    - hyperparams_path, tree_path (str): lần lượt là đường dẫn tới tập tin lưu các siêu tham số
    và lưu cấu trúc cây của mô hình.

    Đầu ra:
    - n_estimators (int): số (nhóm) cây thành phần của mô hình.
    - n_classes (int): số phân lớp đã huấn luyện
    - features (numpy.array): mảng thể hiện đặc trưng được xét tại nút trong cây.
    - values (numpy.array): mảng thể hiện giá trị phân nhánh hoặc nút lá.
    '''
    # mở tập tin, phân tích cú pháp và lưu thông tin vào từ điển
    with open(hyperparams_path) as f:
        hyperparams = json.load(f)

    with open(tree_path) as f:
        tree_list = json.load(f)

    # truy xuất các siêu tham số
    attributes = json.loads(hyperparams['learner']['attributes']['scikit_learn'])
    n_estimators = attributes['n_estimators']
    n_classes = attributes['n_classes_']
    max_depth = attributes['max_depth']
    if max_depth is None:
        max_depth = 6

    # xác định số nút tối đa của cây thành phần và tạo mảng.
    nrows = len(tree_list)
    ncols = sum((2**i for i in range(max_depth+1)))
    features = empty((nrows, ncols), int32)
    values = empty((nrows, ncols), float64)

    # đọc và nạp thông tin các cây vào mảng
    for feat_arr, val_arr, tree in zip(features, values, tree_list):
        load_tree(feat_arr, val_arr, tree)

    return n_estimators, n_classes, features, values

@njit
def traverse_tree(x:array, features_arr:array, values_arr:array):
    '''
    Hàm duyệt cây quyết định và đưa ra dự đoán chon một đối tượng.

    Đầu vào:
    - x (numpy.array): đối tượng dự đoán.
    - features_arr, values_arr (numpy.array): mảng đặc trưng và giá trị của cây quyết định.

    Đầu ra:
    - value (float): giá trị dự đoán.
    '''
    idx = 0

    # kiểm tra nút lá
    while features_arr[idx] != -1:
        # so sánh giá trị đặc trưng của đối tượng và giá trị phân nhánh
        # duyệt tới nút con phù hợp.
        if x[features_arr[idx]] - values_arr[idx] < 0:
            idx = 2 * idx + 1
        else:
            idx = 2 * idx + 2

    return values_arr[idx]

@njit
def predict_proba(X:array, features_arr:array, values_arr:array, n_classes:int32):
    '''
    Hàm dự đoán tập dữ liệu trên mô hình XGBoost được đọc.

    Đầu vào:
    - X (numpy.array): tập dữ liệu dự đoán.
    - features_arr, values_arr (numpy.array): mảng đặc trưng và giá trị của cây quyết định.
    - n_classes (int): số phân lớp được huấn luyện.

    Đầu ra:
    - y_pred (numpy.array): mảng dự đoán xác suất thuộc về các phân lớp của các đối tượng
    '''
    y_pred = zeros((X.shape[0], n_classes))

    for i, x in enumerate(X):
        for j, (f_arr, v_arr) in enumerate(zip(features_arr, values_arr)):
            y_pred[i, j % n_classes] += traverse_tree(x, f_arr, v_arr)

    return softmax(y_pred)

In [ ]:
n_estimators, n_classes, features, values = load_model('/content/xgb_hyperparams.json', '/content/xgb_tree.json')
X_test = np.load('/content/X_test.npy')

In [ ]:
xgblib_model = XGBClassifier(objective='multi:softmax', use_label_encoder=False,
                                                    num_class=n_classes, n_estimators=n_estimators, learning_rate=0.2)

xgblib_model.load_model('/content/xgb_hyperparams.json')

In [ ]:
xgblib_pred = xgblib_model.predict(X_test)

In [ ]:
%%time
xgb_pred = np.argmax(predict_proba(X_test, features, values, n_classes), 1)

CPU times: user 45.4 ms, sys: 750 µs, total: 46.1 ms
Wall time: 46 ms


In [ ]:
np.count_nonzero(xgblib_pred != xgb_pred)

### Độ chính xác & thời gian

| Hàm | Sai số |Thời gian |Ghi chú |
|:--------:|:-----------:|:-----------:|:-----------:|
|rgb2gray  |  1.43e-06 |  413 ms |  | 
| GBR2HSV  | 0.025|65.1 ms|  Không sử dụng| 
| GaussianBlur  |  108.53 | 2.36 s |  Không sử dụng | 
| Histogram  | 0 | 343 ms|  | 
| huMoments  | 2.32e-23 | 2.27 s | | 
| duyệt cây  | |  46 ms|   | 
| zipImage  |  | 996 ms |   | 
| joinNeiboorPixel  |  | 767 ms |   | 

**Ghi chú**: ô trống là tự cài đặt từ đầu.


# Tài liệu tham khảo
- https://towardsdatascience.com/using-nmf-to-classify-companies-a77e176f276f
- https://github.com/jonasrothfuss/fishervector
- https://hal.inria.fr/hal-00779493/file/RR-8209.pdf
- https://viblo.asia/p/sift-scale-invariant-feature-transform-huan-luyen-mo-hinh-cho-cac-bai-toan-phan-loai-924lJqJaZPM
- https://xgboost.readthedocs.io/en/stable/
- https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663
- https://medium.com/analytics-vidhya/what-makes-xgboost-so-extreme-e1544a4433bb